In [1]:
import numpy as np
from sklearn import svm

from src.utils import load_and_regularize_covariance_matrices, calculate_covariance_means, project_to_tangent_and_triu, evaluate_metrics_repeatedly, reduce_spd_dataset
from src.covariance_means import generalized_eigenvalue_covariance_mean

%load_ext autoreload
%autoreload 2

In [2]:
c_dataset = load_and_regularize_covariance_matrices(deep_features_type='covs_DN20', group_type='CG')
c_dataset_covs = calculate_covariance_means(c_dataset) #Generalized eigenvalue covariance mean
c_dataset_covs = reduce_spd_dataset(c_dataset_covs, 3)
c_dataset_vectors = project_to_tangent_and_triu(c_dataset_covs)

pk_dataset = load_and_regularize_covariance_matrices(deep_features_type='covs_DN20', group_type='PG')
pk_dataset_covs = calculate_covariance_means(pk_dataset)
pk_dataset_covs = reduce_spd_dataset(pk_dataset_covs, 3)
pk_dataset_vectors = project_to_tangent_and_triu(pk_dataset_covs)

Patients [Tangent space]: 100%|██████████| 11/11 [00:00<00:00, 93.25it/s]


In [3]:
# Concatenate control and Pk
dataset_vectors = np.concatenate([c_dataset_vectors, pk_dataset_vectors], axis=0) # (22=parkinsons + 22=control, num_videos, num_features = Tangent space flattened mean covariances)
dataset_labels = np.concatenate([np.zeros(c_dataset_vectors.shape[0]), np.ones(pk_dataset_vectors.shape[0])])

print(f"Dataset vectors shape: {dataset_vectors.shape}")

Dataset vectors shape: (22, 8, 6)


In [4]:
# Evaluate metrics
evaluate_metrics_repeatedly(dataset_vectors = dataset_vectors,
                      dataset_labels = dataset_labels,                      
                      estimator = svm.SVC())

Repetitions: 100%|██████████| 1/1 [00:00<00:00, 13.98it/s]


,Accuracy,std Accuracy,Precision,std Precision,Recall,std Recall,F1 Score,std F1 Score
0,0.9375,0.158786,0.931818,0.171587,0.900568,0.250468,0.911422,0.223095
